In [110]:
from knapsackgym import KnapsackEnv

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import copy
from typing import List, Callable, Optional, Union, Tuple, Dict, Any
from tqdm import tqdm


# Actor Critic Model

## Policy Network (default: 64 x 64 (2 hidden layers), sigmoid)

In [111]:
class PolicyNetwork(nn.Module):
    """Policy network for the A2C algorithm to solve the Knapsack Problem"""
    
    def __init__(
        self, 
        input_size: int, 
        output_size: int, 
        hidden_size: int = 64, 
        num_layers: int = 2, 
        activation: str = "sigmoid"
    ):
        """
        Initialize the policy network with customizable architecture.
        
        Args:
            input_size (int): Size of the input vector (2N + 4)
            output_size (int): Size of the output vector (N)
            hidden_size (int): Size of the hidden layers (default: 64)
            num_layers (int): Number of hidden layers (default: 2)
            activation (str): Activation function to use (default: "sigmoid")
                              Options: "sigmoid", "relu", "tanh", "leaky_relu"
        """
        super(PolicyNetwork, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.activation_funcs:Dict[str, Callable] = {
            "sigmoid":F.sigmoid, 
            "relu": F.relu, 
            "tanh": F.tanh, 
            "leaky_relu": F.leaky_relu
        }

        # Set activation function
        if activation not in self.activation_funcs:
            raise ValueError(f"Unsupported activation function: {activation}")
        
        self.activation = self.activation_funcs[activation]
        
        # Create layers dynamically
        self.layers = nn.ModuleList()
        
        # Input layer
        self.layers.append(nn.Linear(input_size, hidden_size))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
            
        # Output layer
        self.layers.append(nn.Linear(hidden_size, output_size))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        
        Args:
            x (torch.Tensor): Input tensor
            
        Returns:
            torch.Tensor: Output logits
        """
        # Pass through all layers except the last one with activation
        for i in range(len(self.layers) - 1):
            x = self.activation(self.layers[i](x))
        
        # No activation on the output layer
        return self.layers[-1](x)
    
    def get_action(self, state: Union[List[float], torch.Tensor, np.ndarray], 
                  available_actions: List[int]) -> int:
        """
        Get action according to policy.
        
        Args:
            state (Union[List[float], torch.Tensor, numpy.ndarray]): Current state
            available_actions (List[int]): List of available actions
            
        Returns:
            int: Chosen action index
        """
        with torch.no_grad():
            # Convert state to tensor if it's not already
            if not isinstance(state, torch.Tensor):
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
            else:
                state_tensor = state.unsqueeze(0) if state.dim() == 1 else state
                
            action_probs = F.softmax(self.forward(state_tensor), dim=1)
            
            # Mask unavailable actions
            action_mask = torch.zeros_like(action_probs)
            for action in available_actions:
                if action < action_mask.shape[1]:
                    action_mask[0, action] = 1
            
            masked_probs = action_probs * action_mask
            
            # If all actions are masked, choose randomly from available actions
            if torch.sum(masked_probs) == 0:
                return random.choice(available_actions)
            
            # Normalize probabilities
            masked_probs = masked_probs / torch.sum(masked_probs)
            
            # Sample action from the masked probability distribution
            action = torch.multinomial(masked_probs, 1).item()
            
            return action

## Value Network (default: 64 x 64 (2 hidden layers), sigmoid)

In [112]:
class ValueNetwork(nn.Module):
    """Value network for the A2C algorithm to estimate state values"""
    
    def __init__(
        self, 
        input_size: int, 
        hidden_size: int = 64, 
        num_layers: int = 2, 
        activation: str = "sigmoid"
    ):
        """
        Initialize the value network with customizable architecture.
        
        Args:
            input_size (int): Size of the input vector (2N + 4)
            hidden_size (int): Size of the hidden layers (default: 64)
            num_layers (int): Number of hidden layers (default: 2)
            activation (str): Activation function to use (default: "sigmoid")
                             Options: "sigmoid", "relu", "tanh", "leaky_relu"
        """
        super(ValueNetwork, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Dictionary mapping activation function names to their implementations
        self.activation_functions: Dict[str, Callable] = {
            "sigmoid": F.sigmoid,
            "relu": F.relu,
            "tanh": F.tanh,
            "leaky_relu": F.leaky_relu
        }
        
        # Set activation function
        if activation not in self.activation_functions:
            raise ValueError(f"Unsupported activation function: {activation}. "
                            f"Supported options are: {list(self.activation_functions.keys())}")
        
        self.activation = self.activation_functions[activation]
        
        # Create layers dynamically
        self.layers = nn.ModuleList()
        
        # Input layer
        self.layers.append(nn.Linear(input_size, hidden_size))
        
        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
            
        # Output layer - value networks output a single scalar value
        self.layers.append(nn.Linear(hidden_size, 1))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        
        Args:
            x (torch.Tensor): Input tensor
            
        Returns:
            torch.Tensor: Estimated state value
        """
        # Pass through all layers except the last one with activation
        for i in range(len(self.layers) - 1):
            x = self.activation(self.layers[i](x))
        
        # No activation on the output layer for value estimation
        return self.layers[-1](x)


## State Aggregator (Placeholder for now)

In [113]:


# Just temporary for now, should be moved to seperate file
class StateAggregator:
    """
    State aggregator to reduce state space dimension.
    Implementation placeholder as mentioned in requirements.
    """
    
    def __init__(self):
        """Initialize the state aggregator"""
        pass
        
    def aggregate(self, state):
        """
        Aggregate state according to equation (7) as mentioned in pseudocode.
        This is a placeholder implementation.
        
        Args:
            state (numpy.ndarray): Original state
            
        Returns:
            numpy.ndarray: Aggregated state
        """
        # For now, simply return the original state (no aggregation)
        return state

## Main KnapsackDRLSolver

In [114]:


class KnapsackDRLSolver:
    """DRL-based Knapsack Solver using A2C algorithm"""
    
    def __init__(self, env, N=100, use_state_aggregation=False, gamma=0.99, lr_policy=0.001, lr_value=0.001, verbose=True):
        """
        Initialize the DRL solver.
        
        Args:
            env: Gym environment for the knapsack problem
            N (int): Maximum number of items in a problem instance
            use_state_aggregation (bool): Whether to use state aggregation
            gamma (float): Discount factor
            lr_policy (float): Learning rate for policy network
            lr_value (float): Learning rate for value network
        """
        self.env:KnapsackEnv = env
        self.N = N
        self.use_state_aggregation = use_state_aggregation
        self.gamma = gamma
        
        # Define input and output sizes for networks
        input_size = 2 * N + 4  # As specified in the requirements
        output_size = N  # One output per potential item
        
        # Initialize networks
        self.policy_net = PolicyNetwork(input_size, output_size)
        self.value_net = ValueNetwork(input_size)
        
        # Initialize optimizers
        self.policy_optimizer = optim.Adam(self.policy_net.parameters(), lr=lr_policy)
        self.value_optimizer = optim.Adam(self.value_net.parameters(), lr=lr_value)
        
        # Initialize state aggregator if needed
        self.state_aggregator = StateAggregator() if use_state_aggregation else None

        self.verbose = verbose
        
    def process_state(self, state):
        """
        Process state with optional aggregation.
        
        Args:
            state (numpy.ndarray): Original state
            
        Returns:
            numpy.ndarray: Processed state
        """
        if self.use_state_aggregation and self.state_aggregator is not None:
            return self.state_aggregator.aggregate(state)
        return state
    
    def train(self, problem_instances, t_max=None):
        """
        Train the DRL solver on multiple problem instances with progress tracking.
        
        Args:
            problem_instances (List[Dict]): List of problem instances
            t_max (int): Maximum number of training steps
            
        Returns:
            List[float]: Values of solutions for each problem instance
        """
        assert len(problem_instances) is not None or len(problem_instances) > 0

        if t_max is None:
            t_max = 3 * self.N * 10000  # As specified in the pseudocode
        
        val = [0] * len(problem_instances)  # Initialize solution values
        
        print(f"Training on {len(problem_instances)} KP Instances, with N={self.N}, t_max={t_max}")
        for t in range(t_max):

            # Select a problem instance (line 6 in pseudocode)
            # P_idx = np.random.randint(0, len(problem_instances))
            P_idx = t % len(problem_instances)
            P = problem_instances[P_idx]
            
            assert len(P['values'] )<= self.N, f"Problem Instance has too many items. KnapsackEnv is configuered to accept no more than <= {self.N}."

            # Change the environment to use this problem instance
            self.env.change_problem_instance(P)
            
            # Reset environment and get initial state
            state = self.env.reset()
            
            # Initialize for this episode
            done = False
            ow = 0  # Total weight of selected items
            ov = 0  # Total value of selected items
            
            # Create a copy of the problem instance P for modification
            n_P_prime = len(P['values'])
            W_P_prime = P['capacity']
            
            # Store states, actions, rewards for batch update
            states:List[np.ndarray] = []
            actions:List[int] = []
            rewards:List[float] = []
            next_states:List[np.ndarray] = []
            dones:List[bool] = []
            
            # Track episode rewards
            episode_rewards:List[float] = []
            
            # Solve the knapsack problem for this instance
            while ow < W_P_prime and n_P_prime > 0 and not done:
                # Process state if needed
                processed_state = self.process_state(state)
                states.append(processed_state)
                
                # Get available actions (indices of remaining items)
                available_actions = list(range(len(self.env.items)))
                
                # Get action according to policy (line 12 in pseudocode)
                action = self.policy_net.get_action(processed_state, available_actions)
                actions.append(action)
                
                # Take action and observe reward and next state
                next_state, reward, done, info = self.env.step(action)
                rewards.append(reward)
                episode_rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)
                
                # Check if item fits in knapsack (line 13 in pseudocode)
                if action < n_P_prime and info['current_weight'] - ow <= W_P_prime:
                    # Update totals (line 14 in pseudocode)
                    ow = info['current_weight']
                    ov = info['current_value']
                    W_P_prime = P['capacity'] - ow
                
                # Update P_prime (line 16 in pseudocode)
                n_P_prime -= 1
                
                # Update state
                state = next_state
            
            # Update parameters using collected trajectories
            self.update_parameters(states, actions, rewards, next_states, dones)
            
            if self.verbose and t % 1000 == 0:
                print(f"Iteration [{t}/{t_max}], Training KP Instance {P_idx}, Reward: {sum(rewards) / len(rewards)}")


            # Update best value if needed (lines 20-22 in pseudocode)
            if ov > val[P_idx]:
                val[P_idx] = ov
            
        return val
    
    def update_parameters(self, states, actions, rewards, next_states, dones):
        """
        Update policy and value network parameters using collected trajectories.
        
        Args:
            states (List[numpy.ndarray]): List of states
            actions (List[int]): List of actions
            rewards (List[float]): List of rewards
            next_states (List[numpy.ndarray]): List of next states
            dones (List[bool]): List of done flags
        """
        # Convert to tensors
        states_tensor = torch.FloatTensor(states)
        actions_tensor = torch.LongTensor(actions)
        rewards_tensor = torch.FloatTensor(rewards)
        next_states_tensor = torch.FloatTensor(next_states)
        dones_tensor = torch.FloatTensor(dones)
        
        # Compute state values
        state_values = self.value_net(states_tensor).squeeze()
        next_state_values = self.value_net(next_states_tensor).squeeze()
        
        # Compute returns and advantages for A2C
        returns = []
        advantages = []
        R = 0
        
        # Compute returns with bootstrapping
        for i in reversed(range(len(rewards))):
            R = rewards[i] + self.gamma * R * (1 - dones[i])
            returns.insert(0, R)
        
        returns_tensor = torch.FloatTensor(returns)
        
        # Compute advantage = returns - state_values
        advantages = returns_tensor - state_values.detach()
        
        # Update value network (equation 3 in pseudocode)
        value_loss = F.mse_loss(state_values, returns_tensor)
        self.value_optimizer.zero_grad()
        value_loss.backward()
        self.value_optimizer.step()
        
        # Update policy network (equation 2 in pseudocode)
        policy_output = self.policy_net(states_tensor)
        action_probs = F.softmax(policy_output, dim=1)
        
        # Extract probabilities of chosen actions
        action_log_probs = F.log_softmax(policy_output, dim=1)
        selected_action_log_probs = action_log_probs.gather(1, actions_tensor.unsqueeze(1)).squeeze()
        
        # Policy gradient loss
        policy_loss = -(selected_action_log_probs * advantages).mean()
        
        self.policy_optimizer.zero_grad()
        policy_loss.backward()
        self.policy_optimizer.step()
    
    def solve(self, problem_instance):
        """
        Solve a single knapsack problem instance using the trained policy.
        
        Args:
            problem_instance (Dict): A problem instance
            
        Returns:
            Tuple[float, List[int]]: Total value and list of selected item indices
        """
        # Set environment to use this problem instance
        self.env.change_problem_instance(problem_instance)
        
        # Reset environment and get initial state
        state = self.env.reset()
        
        done = False
        total_value = 0
        total_weight = 0
        selected_items = []
        
        while not done:
            # Process state if needed
            processed_state = self.process_state(state)
            
            # Get available actions (indices of remaining items)
            available_actions = list(range(len(self.env.items)))
            
            if not available_actions:
                break
                
            # Get action according to policy
            action = self.policy_net.get_action(processed_state, available_actions)
            
            # Get original item index (before any removal)
            original_item_idx = self.env.items[action][2]
            
            # Take action and observe reward and next state
            next_state, reward, done, info = self.env.step(action)
            
            # If item was added (positive reward means item fit)
            if reward > 0:
                selected_items.append(original_item_idx)
            
            # Update value and weight
            total_value = info['current_value']
            total_weight = info['current_weight']
            
            # Update state
            state = next_state
        
        return total_value, selected_items

# Training Loop

In [115]:
def solve_knapsack_dp(problem: Dict[str, Any]) -> Tuple[float, List[int]]:
    """
    Solves the 0-1 Knapsack Problem using Dynamic Programming.
    
    Args:
        problem (Dict): A dictionary containing:
            - 'values': List of item values
            - 'weights': List of item weights
            - 'capacity': Knapsack capacity
    
    Returns:
        Tuple[float, List[int]]: A tuple containing:
            - The maximum value achievable
            - A list of indices of the selected items
    """
    values = problem['values']
    weights = problem['weights']
    capacity = problem['capacity']
    n = len(values)
    
    # Create DP table: rows are items, columns are capacities (0 to W)
    dp = np.zeros((n + 1, capacity + 1), dtype=float)
    
    # Fill the DP table
    for i in range(1, n + 1):
        for w in range(capacity + 1):
            # If item i-1 can fit in knapsack of capacity w
            if weights[i-1] <= w:
                # Max of (including this item, excluding this item)
                dp[i, w] = max(values[i-1] + dp[i-1, w-weights[i-1]], dp[i-1, w])
            else:
                # Cannot include this item
                dp[i, w] = dp[i-1, w]
    
    # Backtrack to find the selected items
    selected_items = []
    w = capacity
    for i in range(n, 0, -1):
        # If this item was included
        if dp[i, w] != dp[i-1, w]:
            selected_items.append(i-1)  # Item index is i-1
            w -= weights[i-1]
    
    # Reverse the list to get items in original order
    selected_items.reverse()
    
    return dp[n, capacity], selected_items

def train_knapsack_solver(env, problem_instances, N=100, use_state_aggregation=False, gamma=0.99, 
                         lr_policy=0.001, lr_value=0.001, t_max=None, verbose=True, compute_optimal=True):
    """
    Train a DRL-based knapsack solver on multiple problem instances.
    
    Args:
        env: Gym environment for knapsack problem
        problem_instances (List[Dict]): List of problem instances
        N (int): Maximum number of items in a problem instance
        use_state_aggregation (bool): Whether to use state aggregation
        gamma (float): Discount factor
        lr_policy (float): Learning rate for policy network
        lr_value (float): Learning rate for value network
        t_max (int): Maximum number of training steps
        
    Returns:
        Tuple[KnapsackDRLSolver, List[float]]: Trained solver and solution values
    """
    # Initialize solver
    solver = KnapsackDRLSolver(
        env=env,
        N=N,
        use_state_aggregation=use_state_aggregation,
        gamma=gamma,
        lr_policy=lr_policy,
        lr_value=lr_value,
        verbose=verbose
    )
    
    # Train solver
    solution_values = solver.train(problem_instances, t_max)
    
    optimal_sols = [None] * len(problem_instances)

    if compute_optimal:
        for i, instance in enumerate(problem_instances):
            opt_sol, opt_items = solve_knapsack_dp(instance)
            optimal_sols[i] = opt_sol

    return solver, solution_values, optimal_sols


def evaluate_knapsack_solver(solver:KnapsackDRLSolver, test_instances:list[dict]):
    """
    Evaluate the trained solver on test instances.
    
    Args:
        solver (KnapsackDRLSolver): Trained solver
        test_instances (List[Dict]): List of test problem instances
        
    Returns:
        List[Dict]: Evaluation results for each test instance
    """
    results = []
    
    for i, instance in enumerate(test_instances):
        print(f"Training on KP Instance {i}")

        # Solve instance
        value, selected_items = solver.solve(instance)
        
        # Calculate actual total weight
        total_weight = sum(instance['weights'][idx] for idx in selected_items)
        
        # Check if solution respects capacity constraint
        is_valid = total_weight <= instance['capacity']

        optimal_value, optimal_items = solve_knapsack_dp(instance)
        
        # Store results
        results.append({
            'instance_idx': i,
            'total_value': value,
            'total_weight': total_weight,
            'capacity': instance['capacity'],
            'selected_items': selected_items,
            'is_valid': is_valid,
            "optimal_sol": optimal_value,
            "optimal_items": optimal_items,
            "optimality_ratio": value /  optimal_value 
        })
    
    return results

## Testing stuff

In [119]:
# Example usage
if __name__ == "__main__":
    
    # Create problem instances
    problem_instances = [
        {
            'values': [10, 5, 15, 7, 6, 18, 3, 20],
            'weights': [2, 3, 5, 7, 1, 4, 1, 5],
            'capacity': 15
        },
        {
            'values': [20, 30, 15, 25, 10, 14, 2, 6],
            'weights': [5, 10, 8, 12, 4, 2, 5, 20],
            'capacity': 20
        }
    ]

    N = 20
    t_max = 10000
    env:KnapsackEnv = KnapsackEnv(problem_instance=None, N=N)

    # Train solver
    solver, solution_values, optimal_sol = train_knapsack_solver(
        env=env,
        problem_instances=problem_instances,
        N=N,
        use_state_aggregation=False,
        t_max=t_max, # Set a smaller value for testing
        verbose=True,
        compute_optimal=True
    )
    
    print("Trained solution values:", solution_values)
    print("Optimal solution values:", optimal_sol)
    
    # Evaluate on test instances
    test_instances = [
        {
            'values': [10, 20, 30],
            'weights': [5, 10, 15],
            'capacity': 20
        }
    ]
    
    solver.verbose = False
    results = evaluate_knapsack_solver(solver, test_instances)
    print("Evaluation results:", results)

Training on 2 KP Instances, with N=20, t_max=10000
Iteration [0/10000], Training KP Instance 0, Reward: 0.19259259259259257


/var/folders/66/lykx91p150q4wl3w7w5ypwdh0000gn/T/ipykernel_42447/720289629.py:189: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  value_loss = F.mse_loss(state_values, returns_tensor)


Iteration [1000/10000], Training KP Instance 0, Reward: 0.29166666666666663
Iteration [2000/10000], Training KP Instance 0, Reward: 0.29166666666666663
Iteration [3000/10000], Training KP Instance 0, Reward: 0.29166666666666663
Iteration [4000/10000], Training KP Instance 0, Reward: 0.29166666666666663
Iteration [5000/10000], Training KP Instance 0, Reward: 0.32499999999999996
Iteration [6000/10000], Training KP Instance 0, Reward: 0.32499999999999996
Iteration [7000/10000], Training KP Instance 0, Reward: 0.32499999999999996
Iteration [8000/10000], Training KP Instance 0, Reward: 0.32499999999999996
Iteration [9000/10000], Training KP Instance 0, Reward: 0.32499999999999996
Trained solution values: [54.0, 64.0]
Optimal solution values: [59.0, 64.0]
Training on KP Instance 0
Evaluation results: [{'instance_idx': 0, 'total_value': 30.0, 'total_weight': 15, 'capacity': 20, 'selected_items': [0, 1], 'is_valid': True, 'optimal_sol': 40.0, 'optimal_items': [0, 2], 'optimality_ratio': 0.75}]